# Followers fix

## Setup

In [1]:
# Import modules
import pandas as pd

from collections import Counter
from tqdm import tqdm


In [ ]:
# Import the data
followers_df = pd.read_csv('/Users/julienmbarki/Documents/Doctorat/Publications/Article 2/Data/Code/data_management/db_soundcharts/editorial_tracks_followers_23-24.csv')
main_df = pd.read_csv('/Users/julienmbarki/Documents/Doctorat/Publications/Article 2/Data/Code/data_management/editorial_playlists_23-24_final.csv')
display(followers_df)


## Clean followers data

In [ ]:
# Collapse to weekly level playlists data
# Convert date column to datetime
followers_df['playlist_followers_date'] = pd.to_datetime(followers_df['playlist_followers_date'])

# Extract year and week number
followers_df['year'] = followers_df['playlist_followers_date'].dt.isocalendar().year
followers_df['week'] = followers_df['playlist_followers_date'].dt.isocalendar().week
followers_df['weekday'] = followers_df['playlist_followers_date'].dt.weekday  # Monday=0, ..., Sunday=6

# Sort data to ensure selection priority
followers_df.sort_values(by=['playlist_id', 'year', 'week', 'weekday'], ascending=[True, True, True, True], inplace=True)

# Keep only one entry per playlist_id per week, prioritizing Friday (4)
df_weekly = followers_df.groupby(['playlist_id', 'year', 'week']).apply(lambda x: x[x['weekday'] == 4].iloc[:1] if (x['weekday'] == 4).any() else x.iloc[:1])

# Reset index
df_weekly = df_weekly.droplevel(0).reset_index(drop=True)

# Display result
display(df_weekly)


In [ ]:
# Merge the data and append it to the correct week
main_df['collection_date'] = pd.to_datetime(main_df['collection_date'])
main_df['year'] = main_df['collection_date'].dt.isocalendar().year
main_df['week'] = main_df['collection_date'].dt.isocalendar().week

# Select only the columns you need from df_weekly (for example, 'followers')
df_weekly_subset = df_weekly[['playlist_id', 'year', 'week', 'playlist_followers']]

# Left merge on playlist_id, year, and week
merged_df = pd.merge(main_df, df_weekly_subset, how='left', on=['playlist_id', 'year', 'week'])

# Display result
display(merged_df)
